In [1]:
from jesse.helpers import date_to_timestamp
import lightgbm as lgb
import pandas as pd
import numpy as np
from strategies.BinanceBtcDeapV1Voting.models.config import FEAT_L4, FEAT_FRACDIFF, DeepSSMContainer, LGSSMContainer

df_feat_raw = pd.read_parquet("data/feat_hard_L4.parquet")

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


PyTorch configured: device=cpu, dtype=torch.float32


In [ ]:
model_deep_ssm = DeepSSMContainer()
model_lg_ssm = LGSSMContainer()

In [ ]:
df_feat_raw = pd.read_parquet("data/feat_hard_L4.parquet")

In [ ]:
df_deep_ssm = model_deep_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_feat_raw[FEAT_FRACDIFF])
df_feat_l4_full = pd.concat([df_deep_ssm, df_lg_ssm, df_feat_raw], axis=1)[FEAT_L4]

test_features = df_feat_l4_full[
    (df_feat_l4_full.index.to_numpy() >= date_to_timestamp("2025-03-01")) &
    (df_feat_l4_full.index.to_numpy() < date_to_timestamp("2025-06-30"))
    ]

candles = np.load("data/bar_deap_v1.npy")
timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

volume = candles[:, 5][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")

assert len(volume) == len(datetimes)

model_l4 = lgb.Booster(model_file="strategies/BinanceBtcDeapV1Voting/models/model_l4.txt")
res = model_l4.predict(test_features)
assert len(res) == len(datetimes)

In [20]:
from research.plot import analyze_confidence_slices

analyze_confidence_slices(
    datetimes,
    res,
    volume,
)

已生成: conf_0.98_0.99_ratio_0.0593.jpg (置信度区间: [0.98, 0.99], 样本占比: 5.93%)
已生成: conf_0.97_0.98_ratio_0.0750.jpg (置信度区间: [0.97, 0.98], 样本占比: 7.50%)
已生成: conf_0.96_0.97_ratio_0.0419.jpg (置信度区间: [0.96, 0.97], 样本占比: 4.19%)
已生成: conf_0.95_0.96_ratio_0.0471.jpg (置信度区间: [0.95, 0.96], 样本占比: 4.71%)
已生成: conf_0.94_0.95_ratio_0.0297.jpg (置信度区间: [0.94, 0.95], 样本占比: 2.97%)
已生成: conf_0.93_0.94_ratio_0.0192.jpg (置信度区间: [0.93, 0.94], 样本占比: 1.92%)
已生成: conf_0.92_0.93_ratio_0.0122.jpg (置信度区间: [0.92, 0.93], 样本占比: 1.22%)
已生成: conf_0.91_0.92_ratio_0.0122.jpg (置信度区间: [0.91, 0.92], 样本占比: 1.22%)
已生成: conf_0.90_0.91_ratio_0.0105.jpg (置信度区间: [0.90, 0.91], 样本占比: 1.05%)
已生成: conf_0.89_0.90_ratio_0.0122.jpg (置信度区间: [0.89, 0.90], 样本占比: 1.22%)
已生成: conf_0.88_0.89_ratio_0.0140.jpg (置信度区间: [0.88, 0.89], 样本占比: 1.40%)
已生成: conf_0.87_0.88_ratio_0.0192.jpg (置信度区间: [0.87, 0.88], 样本占比: 1.92%)
已生成: conf_0.86_0.87_ratio_0.0140.jpg (置信度区间: [0.86, 0.87], 样本占比: 1.40%)
已生成: conf_0.85_0.86_ratio_0.0087.jpg (置信度区间: [0.85, 0.86], 样本占比:

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data_1 = pd.read_csv('model_l4_scores.csv')          ############   测试文件

data_1_size = data_1.shape[0]     ###### 测试数据行数  ###############
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000     ###### 总资金 ###################

In [2]:
ft = 0.01   ### 0.1, 0.05, 0.01

if ft == 0.1:
    da = 1
    xiao = 0.9
    z = 5

if ft == 0.05:
    da = 1
    xiao = 0.95
    z = 10
    
if ft == 0.01:
    da = 1
    xiao = 0.99
    z = 50


for i in range(z):        

    data_1['final'] = 0

    mask = (data_1['score'] < da) & (data_1['score'] >= xiao)
    data_1['final'] = np.where(mask, data_1['volume'], 0)
    
    true_count = mask.sum() / data_1_size 
    
    ### 更新置信度
    da = round((da - ft), 2)
    xiao = round((xiao - ft), 2)

    data_1['high'] = data_1['final'].cumsum()
    data_1['open'] = rrr * data_1['high'] / m

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1['eob'], data_1['open'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))   
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(da) + "_" + str(true_count.round(2)) + ".jpg")   
    plt.close()